In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from skimage.transform import resize
from tensorflow.keras.regularizers import l2

def load_data():
    # Load training data and labels from numpy files
    data = np.load('X.npy')
    labels = np.load('labels.npy') 
    # Reshape and resize data for the model
    data_reshaped = data.T.reshape((-1, 300, 300, 3))
    resized_data = np.array([resize(img, (100, 100, 3), anti_aliasing=True) for img in data_reshaped])
    # Normalize pixel values if necessary
    if resized_data.max() > 1.0:
        data_normalized = resized_data.astype(np.float32) / 255.0
    else:
        data_normalized = resized_data
    return data_normalized, labels

def build_model():
    # Define the CNN architecture
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', input_shape=(100, 100, 3)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(512, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        Dropout(0.5),
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        Dense(9, activation='softmax')
    ])
    return model

def random_brightness(image):
    # Apply random brightness to an image
    image = tf.expand_dims(image, 0)
    image = tf.image.random_brightness(image, max_delta=0.1)
    image = tf.squeeze(image)
    return image

def adjust_contrast(img):
    # Randomly adjust contrast of an image
    contrast_factor = tf.random.uniform([], 0.9, 1.1)
    return tf.image.adjust_contrast(img, contrast_factor)

def preprocess_image(img):
    # Preprocess images by applying random brightness and contrast adjustments
    img = random_brightness(img)
    img = adjust_contrast(img)
    img = tf.clip_by_value(img, 0.0, 1.0)
    return img

def train():
    # Load data and labels
    data, labels = load_data()
    # Define image data generator for data augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest',
        preprocessing_function=preprocess_image
    )
    # Build, compile and train the model
    model = build_model()
    model.compile(optimizer=Adam(learning_rate=0.0004), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Note: No train-validation split here, using full data for training
    train_generator = datagen.flow(data, labels, batch_size=32)
    model.fit(train_generator, epochs=35)

    # Save final model to a file
    model.save('final_model.h5')

if __name__ == "__main__":
    train()
